In [1]:
from random import choices
from nltk.tokenize import regexp_tokenize, word_tokenize
from tqdm import tqdm
import gc 
import re
import math

In [2]:
# Read file and tokenize
file = open("label_train.txt", "r", encoding="utf8")
a = file.read()
file.close()

train_label = word_tokenize(a)

del a
gc.collect()

print(len(train_label))
print("finish loading train_label!")

7108
finish loading train_label!


In [3]:
train_arr = []
with open('text_train.txt', encoding="utf8") as train_file:
    for line in train_file:
        word_arr = []
        no_punc = re.sub(r'[^\w\s]', '', line)
        word_arr = word_tokenize(no_punc)
        train_arr.append(word_arr)
train_file.close()

print(len(train_arr))
# print(train_arr[0])


7108


In [4]:
#Count words and vocabulary
total_vocab = {}
pos_word = {}
neg_word = {}
pos_line = 0
neg_line = 0
count_pos_word = 0
count_neg_word = 0
count_vocab = 0
for line in train_arr:
    for word in line:
        if word not in total_vocab:
            total_vocab[word] = 1
        else:
            total_vocab[word] += 1

# print(len(total_vocab))
count_vocav = len(total_vocab)

for i in range (len(train_arr)):
    if train_label[i] == "1":
        #positive feedback
        pos_line += 1
        for word in train_arr[i]:
            count_pos_word += 1
            if word not in pos_word:
                pos_word[word] = 1
            else:
                pos_word[word] += 1
                
    elif train_label[i] == "0":
        #negative feedback
        neg_line += 1
        for word in train_arr[i]:
            count_neg_word += 1
            if word not in neg_word:
                neg_word[word] = 1
            else:
                neg_word[word] += 1

pos_lin_prob = pos_line / (pos_line + neg_line)
neg_lin_prob = neg_line / (pos_line + neg_line)

In [5]:
def user_interface():
    print("Welcome! please type a review and we will judge if it's positive or negative")
    user_input = input()
    no_punc = re.sub(r'[^\w\s]', '', user_input)
    word_arr = word_tokenize(no_punc)
    result = determine_sentiment(word_arr)
    print (result)
    
def determine_sentiment(word_arr):
    #calculate positive
    total_pos_prob = 0
    total_neg_prob = 0
    
    for word in word_arr:
        if word in total_vocab:
            word_pos_prob = 0
            if word in pos_word:
                word_pos_prob = math.log ((pos_word[word] + 1) / (count_pos_word + count_vocab))
            else:
                word_pos_prob = math.log (1 / (count_pos_word + count_vocab))
            total_pos_prob += word_pos_prob
    
            word_neg_prob = 0
            if word in neg_word:
                word_neg_prob = math.log ((neg_word[word] + 1) / (count_neg_word + count_vocab))
            else:
                word_neg_prob = math.log (1 / (count_neg_word + count_vocab))
            total_neg_prob += word_neg_prob
            
    total_pos_prob += math.log (pos_lin_prob)
    total_neg_prob += math.log (neg_lin_prob)
    
    if total_pos_prob > total_neg_prob:
        return "Postive"
    elif total_pos_prob == total_neg_prob:
        return "Middle"
    else:
        return "Negative"
    
    
        

In [10]:
user_interface()

Welcome! please type a review and we will judge if it's positive or negative
great movie
Postive


In [7]:
# Read file and tokenize
file = open("label_test.txt", "r", encoding="utf8")
a = file.read()
file.close()

test_label = word_tokenize(a)

del a
gc.collect()

print(len(test_label))
print("finish loading test_label!")

3554
finish loading test_label!


In [8]:
test_arr = []
with open('text_test.txt', encoding="utf8") as test_file:
    for line in test_file:
        word_arr = []
        no_punc = re.sub(r'[^\w\s]', '', line)
        word_arr = word_tokenize(no_punc)
        test_arr.append(word_arr)
test_file.close()

print(len(test_arr))
# print(train_arr[0])


3554


In [9]:
predict_arr = []
for line in test_arr:
    result = determine_sentiment(line)
    if result == "Postive":
        predict_arr.append('1')
    elif result == "Middle":
        predict_arr.append('X')
    elif result == "Negative":
        predict_arr.append('0')

    
correct_count = 0
for i in range(len(test_label)):
    if test_label[i] == predict_arr[i]:
        correct_count += 1

print("correct predict rate is: ", correct_count/len(test_label))

correct predict rate is:  0.7625211029825548
